
# OpenAI 라벨링 파이프라인 (gpt-4o-mini, Colab용)

엑셀 파일을 **pandas DataFrame**으로 읽고, 저장해 둔 **프롬프트(.txt)**를 불러와
각 행의 텍스트를 OpenAI 모델에 입력 → **JSON** 결과를 파싱/검증 →
**JSONL/CSV로 저장**까지 한 번에 수행합니다.

> **모델**: `gpt-4o-mini`  
> **입출력**: Excel → DataFrame → OpenAI → JSON → JSONL/CSV


## 1) 환경 설정

In [1]:
# 설치 시 pandas 버전 고정
!pip install --upgrade openai tqdm pandas==2.2.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.3/786.3 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.99.1
    Uninstalling openai-1.99.1:
      Successfully uninstalled openai-1.99.1


In [2]:
# 모듈 임포트
import os, re, json, time, math, textwrap
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

print('pandas', pd.__version__)


pandas 2.2.2



**API 키 설정**  
- 가장 안전한 방법은 세션 환경변수에 넣는 것입니다.  
- Colab 좌측 `🔑 Variables` 또는 아래 셀의 `os.environ`에 직접 설정해 주세요.


In [3]:
os.environ['OPENAI_API_KEY'] = 'sk-'

assert 'OPENAI_API_KEY' in os.environ and len(os.environ['OPENAI_API_KEY']) > 10, "환경변수 OPENAI_API_KEY가 설정되어 있지 않습니다. 위 주석 참고해 설정하세요."
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
MODEL_NAME = "gpt-4o-mini"
MODEL_NAME


'gpt-4o-mini'

## 2) 데이터 & 프롬프트 파일 경로 지정

In [4]:
from google.colab import drive
drive.mount('/content/drive')

EXCEL_PATH = '/content/drive/MyDrive/STEN_cohort_SMC_preprocess.xlsx'
PROMPT_PATH = '/content/drive/MyDrive/프롬프트.txt'

# 출력 파일
OUT_JSONL = '/content/labels.jsonl'   # 한 줄에 하나의 JSON
OUT_CSV   = '/content/labels_merged.csv'

# 엑셀의 컬럼명 설정 (내 파일에 맞게 변경)
ID_COL    = '환자번호'
TEXT_COL  = '검사결과'
print('EXCEL_PATH:', EXCEL_PATH)
print('PROMPT_PATH:', PROMPT_PATH)


Mounted at /content/drive
EXCEL_PATH: /content/drive/MyDrive/STEN_cohort_SMC_preprocess.xlsx
PROMPT_PATH: /content/drive/MyDrive/프롬프트.txt


## 3) 프롬프트 + 엑셀 로딩

In [5]:
# 프롬프트 로딩 (UTF-8 가정)
with open(PROMPT_PATH, 'r', encoding='utf-8') as f:
    system_prompt = f.read()

# 엑셀 로딩
df = pd.read_excel(EXCEL_PATH)
print('Rows:', len(df))
display(df.head(3))


Rows: 4198


,환자번호,검사결과,L1/2,L2/3,L3/4,L4/5,L5/S1
0,7017,"▣검사정보 및 소견_x000D__x000D_L-S SPINE MRI, MINIMAL...",NaN,NaN,NaN,NaN,NaN
1,7017,"▣검사정보 및 소견_x000D__x000D_L-S SPINE MRI, MINIMAL...",NaN,NaN,NaN,NaN,NaN
2,8715,▣ 검사정보 및 소견\n\n3T L-SPINE MRI (NON-CONTRAST)\n...,NaN,NaN,NaN,NaN,NaN


## 4) JSON 파싱/검증 유틸리티

In [6]:
def extract_json(s: str):
    """모델 출력에서 JSON만 깔끔히 추출해 dict로 반환.
    코드블록, 설명 텍스트가 섞여도 regex로 첫 번째 JSON 블럭을 잡습니다.
    """
    if s is None:
        raise ValueError("빈 응답")
    # ```json ... ``` 혹은 { ... } 블록 추출
    patterns = [
        r"""```json\s*(\{[\s\S]*?\})\s*```""",
        r"""```\s*(\{[\s\S]*?\})\s*```""",
        r"""(\{[\s\S]*\})"""
    ]
    for pat in patterns:
        m = re.search(pat, s)
        if m:
            candidate = m.group(1).strip()
            try:
                return json.loads(candidate)
            except json.JSONDecodeError:
                pass
    # 마지막 시도: 전체를 파싱
    return json.loads(s)

def validate_schema(obj: dict, required_keys=None):
    """라벨 스키마 간단 검증: 필수 키 존재 및 bool 타입 확인"""
    required_keys = required_keys or ["L1/2","L2/3","L3/4","L4/5","L5/S1","need_check"]
    for k in required_keys:
        if k not in obj:
            raise ValueError(f"JSON 키 누락: {k}")
        if not isinstance(obj[k], bool):
            raise TypeError(f"키 '{k}'의 값은 bool이어야 합니다: {obj[k]} ({type(obj[k])})")
    return True


## 5) 단일 행 테스트 (샘플 1건)

In [7]:
sample_text = str(df.iloc[0][TEXT_COL])
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": f"""
아래는 요약/정제 전 원문 판독문입니다. 지시문을 엄격히 따르고, **반드시 STRICT JSON만** 출력하세요.

[원문]
{sample_text}
""".strip()}
]

resp = client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
    temperature=0
)
raw = resp.choices[0].message.content
print('--- raw ---\n', raw[:500], '...')
parsed = extract_json(raw)
validate_schema(parsed)
print('\n--- parsed ---\n', json.dumps(parsed, ensure_ascii=False, indent=2))


--- raw ---
 {
    "L1/2": false,
    "L2/3": false,
    "L3/4": false,
    "L4/5": false,
    "L5/S1": false,
    "need_check": false
} ...

--- parsed ---
 {
  "L1/2": false,
  "L2/3": false,
  "L3/4": false,
  "L4/5": false,
  "L5/S1": false,
  "need_check": false
}


## 6) 전체 배치 라벨링

In [8]:
results = []
errors  = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    txt = str(row[TEXT_COL])
    uid = row[ID_COL] if ID_COL in df.columns else i

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": f"""
아래는 요약/정제 전 원문 판독문입니다. 지시문을 엄격히 따르고, **반드시 STRICT JSON만** 출력하세요.

[원문]
{txt}
""".strip()}
    ]
    try:
        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            temperature=0
        )
        raw = resp.choices[0].message.content
        obj = extract_json(raw)
        validate_schema(obj)
        obj['id'] = uid
        results.append(obj)
    except Exception as e:
        errors.append({'index': int(i), 'id': uid, 'error': str(e)})
        continue
    # 너무 빨리 돌 경우 rate-limit 완충
    time.sleep(0.05)

print(f"완료: {len(results)}건, 오류: {len(errors)}건")
if errors:
    print('\n에러 상위 5건 예시:', errors[:5])


100%|██████████| 4198/4198 [1:39:39<00:00,  1.42s/it]

완료: 4176건, 오류: 22건

에러 상위 5건 예시: [{'index': 44, 'id': 222566, 'error': "키 'L1/2'의 값은 bool이어야 합니다: {'central': True, 'foramen': True, 'subarticular': False} (<class 'dict'>)"}, {'index': 494, 'id': 5559708, 'error': "키 'L1/2'의 값은 bool이어야 합니다: {'central': True, 'foramen': True, 'subarticular': False} (<class 'dict'>)"}, {'index': 540, 'id': 6353408, 'error': "키 'L1/2'의 값은 bool이어야 합니다: {'central': False, 'foramen': True, 'subarticular': True} (<class 'dict'>)"}, {'index': 647, 'id': 8119224, 'error': "키 'L1/2'의 값은 bool이어야 합니다: {'central': False, 'foramen': True, 'subarticular': False} (<class 'dict'>)"}, {'index': 1201, 'id': 16752363, 'error': "키 'L1/2'의 값은 bool이어야 합니다: {'central': True, 'foramen': True, 'subarticular': False} (<class 'dict'>)"}]


## 7) 결과 저장 (JSONL & CSV)

In [9]:
# === dict→bool 정규화 + 중복컬럼 제거 병합 + JSONL/CSV 저장 (원샷) ===
# 전제: df, results, ID_COL, OUT_JSONL, OUT_CSV 이 앞에서 이미 정의됨

import os, json
import pandas as pd
from typing import Any, Dict

# 1) 정규화 유틸
def coerce_bool(v: Any) -> bool:
    if isinstance(v, dict):                 # {"central":T, "foramen":F, ...} → 하나라도 True면 True
        return any(bool(x) for x in v.values())
    if isinstance(v, str):
        s = v.strip().lower()
        if s in {"true","1","yes","y"}: return True
        if s in {"false","0","no","n"}: return False
    if isinstance(v, (int, float)): return bool(v)
    return bool(v)

TARGET_KEYS = ["L1/2","L2/3","L3/4","L4/5","L5/S1","need_check"]

def coerce_record(obj: Dict[str, Any]) -> Dict[str, Any]:
    for k in TARGET_KEYS:
        if k in obj:
            obj[k] = coerce_bool(obj[k])
    return obj

# 2) results 정규화 (results_norm 없으면 생성)
if 'results_norm' not in globals():
    assert 'results' in globals(), "results 가 없습니다. 6번(배치 라벨링) 먼저 실행하세요."
    results_norm = [coerce_record(dict(r)) for r in results]

# 3) JSONL 저장 (기존 파일 백업)
if os.path.exists(OUT_JSONL):
    os.rename(OUT_JSONL, OUT_JSONL + ".bak")

with open(OUT_JSONL, 'w', encoding='utf-8') as f:
    for r in results_norm:
        f.write(json.dumps(r, ensure_ascii=False) + '\n')

# 4) CSV 병합 준비
res_df = pd.DataFrame(results_norm)

# id 타입 정리: df[ID_COL]과 res_df['id'] 타입을 맞춰줌 (둘 다 문자열로 통일이 안전)
if ID_COL in df.columns and 'id' in res_df.columns:
    left_id  = df[ID_COL].astype(str)
    right_id = res_df['id'].astype(str)
    df2 = df.copy()
    df2[ID_COL] = left_id
    res_df2 = res_df.copy()
    res_df2['id'] = right_id
else:
    # 혹시라도 빠졌다면 그대로 사용
    df2, res_df2 = df, res_df

# 5) 겹치는 컬럼 제거 후 병합 (id 제외)
overlap = (set(res_df2.columns) & set(df2.columns)) - {'id', ID_COL}
df_clean = df2.drop(columns=list(overlap), errors='ignore')

merged = pd.merge(df_clean, res_df2, left_on=ID_COL, right_on='id', how='left')

# 6) 저장
merged.to_csv(OUT_CSV, index=False, encoding='utf-8-sig')

# 7) 요약 출력
print("저장 완료 ✅")
print("JSONL ->", OUT_JSONL)
print("CSV   ->", OUT_CSV)
print("원본 DF:", df.shape, "| 결과 DF:", res_df.shape, "| 병합 DF:", merged.shape)

# 간단 통계
present_keys = [k for k in TARGET_KEYS if k in merged.columns]
for k in present_keys:
    try:
        print(f"{k} True rate:", round(float(merged[k].mean()), 4))
    except Exception:
        pass

print("\n미리보기:")
display(merged.head(5))


저장 완료 ✅
JSONL -> /content/labels.jsonl
CSV   -> /content/labels_merged.csv
원본 DF: (4198, 7) | 결과 DF: (4176, 7) | 병합 DF: (5573, 9)
L1/2 True rate: 0.0421
L2/3 True rate: 0.142
L3/4 True rate: 0.2316
L4/5 True rate: 0.3363
L5/S1 True rate: 0.2537
need_check True rate: 0.156

미리보기:


,환자번호,검사결과,L1/2,L2/3,L3/4,L4/5,L5/S1,need_check,id
0,7017,"▣검사정보 및 소견_x000D__x000D_L-S SPINE MRI, MINIMAL...",False,False,False,False,False,False,7017
1,7017,"▣검사정보 및 소견_x000D__x000D_L-S SPINE MRI, MINIMAL...",False,False,False,False,False,False,7017
2,7017,"▣검사정보 및 소견_x000D__x000D_L-S SPINE MRI, MINIMAL...",False,False,False,False,False,False,7017
3,7017,"▣검사정보 및 소견_x000D__x000D_L-S SPINE MRI, MINIMAL...",False,False,False,False,False,False,7017
4,8715,▣ 검사정보 및 소견\n\n3T L-SPINE MRI (NON-CONTRAST)\n...,False,False,False,False,False,True,8715


## 8) 간단 검증

In [10]:
# True 라벨 비율 확인
if 'L4/5' in merged.columns:
    print('L4/5 True rate:', merged['L4/5'].mean())
if 'need_check' in merged.columns:
    print('need_check rate:', merged['need_check'].mean())

merged.head(5)


L4/5 True rate: 0.336272040302267
need_check rate: 0.15599136379992803


,환자번호,검사결과,L1/2,L2/3,L3/4,L4/5,L5/S1,need_check,id
0,7017,"▣검사정보 및 소견_x000D__x000D_L-S SPINE MRI, MINIMAL...",False,False,False,False,False,False,7017
1,7017,"▣검사정보 및 소견_x000D__x000D_L-S SPINE MRI, MINIMAL...",False,False,False,False,False,False,7017
2,7017,"▣검사정보 및 소견_x000D__x000D_L-S SPINE MRI, MINIMAL...",False,False,False,False,False,False,7017
3,7017,"▣검사정보 및 소견_x000D__x000D_L-S SPINE MRI, MINIMAL...",False,False,False,False,False,False,7017
4,8715,▣ 검사정보 및 소견\n\n3T L-SPINE MRI (NON-CONTRAST)\n...,False,False,False,False,False,True,8715


## 9) (선택) 중단 후 재시작 / 이어붙이기

In [11]:
# 이전 JSONL에서 이미 처리된 id를 읽어 스킵하는 로직 예시
done_ids = set()
try:
    with open(OUT_JSONL, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                obj = json.loads(line)
                if 'id' in obj:
                    done_ids.add(obj['id'])
            except:
                pass
except FileNotFoundError:
    pass

print('이미 처리된 id 수:', len(done_ids))
# 이후 루프에서 if uid in done_ids: continue 형태로 활용하면 됩니다.


이미 처리된 id 수: 3613
